In [1]:
# Install PyTorch and torchvision
!pip install torch==2.0.1 torchvision==0.15.2

# Install PyTorch Geometric dependencies for CUDA 11.7
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu117.html
!pip install torch-geometric


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
# Download the MovieLens 100K dataset
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

# Unzip the dataset quietly
!unzip -q ml-100k.zip


--2025-03-16 19:24:57--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  12.1MB/s    in 0.4s    

2025-03-16 19:24:57 (12.1 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [6]:
import pandas as pd
import torch

# Load ratings (u.data file: user, item, rating, timestamp)
ratings = pd.read_csv('ml-100k/u.data', sep='\t', header=None,
                      names=['user', 'item', 'rating', 'timestamp'])

# For recommendations, treat ratings >= 4 as positive feedback
ratings = ratings[ratings['rating'] >= 4]

# Adjust IDs to zero-indexed
ratings['user'] = ratings['user'] - 1
ratings['item'] = ratings['item'] - 1

# Use the maximum value + 1 to get the total number of users and items
num_users = int(ratings['user'].max()) + 1
num_items = int(ratings['item'].max()) + 1

print(f'Number of users: {num_users}, Number of items: {num_items}')
ratings.head()


Number of users: 943, Number of items: 1674


,user,item,rating,timestamp
5,297,473,4,884182806
7,252,464,5,891628467
11,285,1013,5,879781125
12,199,221,5,876042340
16,121,386,5,879270459


In [7]:
# Create edge_index tensor (shape: [2, num_edges])
edge_index = torch.tensor(ratings[['user', 'item']].values.T, dtype=torch.long)
print("Edge index shape:", edge_index.shape)


Edge index shape: torch.Size([2, 55375])


In [9]:
from torch_sparse import SparseTensor

def build_norm_adj(num_users, num_items, edge_index):
    # Shift item indices so that users and items share one index space
    edge_index = edge_index.clone()
    edge_index[1] += num_users  # items now start at index num_users

    # Create reverse edges to form a symmetric graph
    row, col = edge_index
    rev_edge_index = torch.stack([col, row], dim=0)
    full_edge_index = torch.cat([edge_index, rev_edge_index], dim=1)

    N = num_users + num_items  # Total nodes
    # Create the sparse adjacency matrix (implicitly with value 1 for every edge)
    adj = SparseTensor(row=full_edge_index[0], col=full_edge_index[1], sparse_sizes=(N, N))

    # Compute degree and its inverse square root
    deg = adj.sum(dim=1).to(torch.float)
    deg_inv_sqrt = deg.pow(-0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

    # Get the COO representation of the sparse tensor
    row, col, val = adj.coo()
    # If no values were set, use 1 for each edge
    if val is None:
        val = torch.ones_like(row, dtype=torch.float)

    # Normalize the values: D^(-1/2) * A * D^(-1/2)
    norm_val = deg_inv_sqrt[row] * val * deg_inv_sqrt[col]
    norm_adj = SparseTensor(row=row, col=col, value=norm_val, sparse_sizes=(N, N))

    return norm_adj

norm_adj = build_norm_adj(num_users, num_items, edge_index)
print("Normalized adjacency matrix built!")


Normalized adjacency matrix built!


In [14]:
import torch.nn as nn

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, emb_size, num_layers):
        super(LightGCN, self).__init__()
        self.num_layers = num_layers
        self.num_users = num_users
        self.num_items = num_items

        # Initialize embeddings for users and items
        self.user_embedding = nn.Embedding(num_users, emb_size)
        self.item_embedding = nn.Embedding(num_items, emb_size)
        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

    def forward(self, norm_adj):
        # Concatenate user and item embeddings into one tensor.
        all_embeddings = torch.cat([self.user_embedding.weight, self.item_embedding.weight], dim=0)
        embeddings_list = [all_embeddings]

        # Propagate embeddings through the graph for num_layers iterations
        for _ in range(self.num_layers):
            # Use the matmul method of SparseTensor instead of torch.sparse.mm
            all_embeddings = norm_adj.matmul(all_embeddings)
            embeddings_list.append(all_embeddings)

        # Average the embeddings from all layers (including the initial ones)
        final_embedding = sum(embeddings_list) / (self.num_layers + 1)

        # Split back into user and item embeddings
        user_emb, item_emb = final_embedding.split([self.num_users, self.num_items])
        return user_emb, item_emb

    def get_score(self, user_emb, item_emb, users, items):
        # Dot product between user and item embeddings as a recommendation score
        u_emb = user_emb[users]
        i_emb = item_emb[items]
        return (u_emb * i_emb).sum(dim=1)

# Set hyperparameters
embedding_size = 64
num_layers = 2

# Create the model instance
model = LightGCN(num_users, num_items, embedding_size, num_layers)
print("LightGCN model created!")


LightGCN model created!


In [15]:
import random
import torch.optim as optim

# Create a dictionary mapping each user to their set of positive items
user_item_dict = ratings.groupby('user')['item'].apply(set).to_dict()

def sample_negative(user):
    # Randomly sample an item that the user has NOT interacted with
    while True:
        neg_item = random.randint(0, num_items - 1)
        if neg_item not in user_item_dict[user]:
            return neg_item

def bpr_loss(user_emb, item_emb, batch):
    users, pos_items, neg_items = [], [], []
    for (user, pos_item) in batch:
        users.append(user)
        pos_items.append(pos_item)
        neg_items.append(sample_negative(user))

    users = torch.tensor(users, dtype=torch.long)
    pos_items = torch.tensor(pos_items, dtype=torch.long)
    neg_items = torch.tensor(neg_items, dtype=torch.long)

    pos_scores = model.get_score(user_emb, item_emb, users, pos_items)
    neg_scores = model.get_score(user_emb, item_emb, users, neg_items)

    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
    return loss

# Prepare the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [17]:
# Create a list of all positive (user, item) pairs
pos_pairs = list(ratings[['user', 'item']].itertuples(index=False, name=None))

num_epochs = 1000
batch_size = 1024  # Adjust the batch size if needed

for epoch in range(1, num_epochs + 1):
    model.train()
    optimizer.zero_grad()

    # Forward pass: compute embeddings from the normalized adjacency matrix
    user_emb, item_emb = model(norm_adj)

    # Randomly sample a batch of positive pairs
    batch = random.sample(pos_pairs, min(batch_size, len(pos_pairs)))

    loss = bpr_loss(user_emb, item_emb, batch)
    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch:03d}, Loss: {loss.item():.4f}")

print("Training finished!")


Epoch 005, Loss: 0.6518
Epoch 010, Loss: 0.6388
Epoch 015, Loss: 0.6248
Epoch 020, Loss: 0.6044
Epoch 025, Loss: 0.5877
Epoch 030, Loss: 0.5766
Epoch 035, Loss: 0.5583
Epoch 040, Loss: 0.5343
Epoch 045, Loss: 0.5146
Epoch 050, Loss: 0.4886
Epoch 055, Loss: 0.4637
Epoch 060, Loss: 0.4655
Epoch 065, Loss: 0.4487
Epoch 070, Loss: 0.4137
Epoch 075, Loss: 0.4049
Epoch 080, Loss: 0.3928
Epoch 085, Loss: 0.3786
Epoch 090, Loss: 0.3649
Epoch 095, Loss: 0.3584
Epoch 100, Loss: 0.3474
Epoch 105, Loss: 0.3443
Epoch 110, Loss: 0.3468
Epoch 115, Loss: 0.3405
Epoch 120, Loss: 0.3349
Epoch 125, Loss: 0.3291
Epoch 130, Loss: 0.3376
Epoch 135, Loss: 0.3228
Epoch 140, Loss: 0.3338
Epoch 145, Loss: 0.3085
Epoch 150, Loss: 0.3026
Epoch 155, Loss: 0.2782
Epoch 160, Loss: 0.3013
Epoch 165, Loss: 0.2980
Epoch 170, Loss: 0.3091
Epoch 175, Loss: 0.3075
Epoch 180, Loss: 0.2985
Epoch 185, Loss: 0.3057
Epoch 190, Loss: 0.3076
Epoch 195, Loss: 0.2712
Epoch 200, Loss: 0.2959
Epoch 205, Loss: 0.2745
Epoch 210, Loss:

In [18]:
def recommend(model, norm_adj, user_id, top_k=10):
    model.eval()
    with torch.no_grad():
        user_emb, item_emb = model(norm_adj)
        # Compute scores for all items for this user
        scores = (user_emb[user_id].unsqueeze(0) * item_emb).sum(dim=1)
        _, top_items = torch.topk(scores, top_k)
    return top_items

# Example: Get top-10 recommendations for user 0
user_id = 0
recommended_items = recommend(model, norm_adj, user_id, top_k=10)
print(f"Top 10 recommendations for user {user_id}:", recommended_items.tolist())


Top 10 recommendations for user 0: [49, 99, 97, 173, 180, 126, 171, 0, 55, 63]
